# Finding things

> Computer languages of the future will be more concerned with goals and less with procedures specified by the programmer.
\--_Marvin Minsky_

Just like with indexing, there seem to be a baffling array of ways to locate items in arrays in APL. As stated in [The Zen of Python](https://zen-of-python.info/): 

> There should be one-- and preferably only one --obvious way to do it.

Aha. About that...

![laugh](https://media.giphy.com/media/JmD9mkDmzvXE7nxy7j/giphy-downsized-medium.gif)

In [1]:
⎕IO ← 0
]box on -s=min -t=tree -f=on
]rows on

Rebuilding user command cache... done

Rebuilding user command cache... done

Was ON -style=min -trains=tree -fns=on

Was OFF

## Equality `=`

So how do you locate where an element resides in an array in APL? Well, the obvious way to do it is to exploit equality and scalar extension. Where is the number 2?

In [11]:
⊢data ← 4 1 22 20 16 10 25 7 18 11 15 2 12 23 9 17 6 14 21 19 3 8 5 13 24
2=data

4 1 22 20 16 10 25 7 18 11 15 2 12 23 9 17 6 14 21 19 3 8 5 13 24

0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0

That surely qualifies in the Zen of Python sense. Hands up every item that equals 2. We can find out the actual index, too, by using _iota-underbar_, `⍸`, which in its monadic form is appropriately enough called _where_:

In [12]:
⍸2=data ⍝ Where is data = 2?

12

As we should expect by now, scalar extension lets scalar functions penetrate any level of nesting:

In [13]:
⊢nested ← (1 2 (3 4 5))(4 (1 2(3 4)))
4=nested

┌───────────┬─────────────┐
│┌─┬─┬─────┐│┌─┬─────────┐│
││1│2│3 4 5│││4│┌─┬─┬───┐││
│└─┴─┴─────┘││ ││1│2│3 4│││
│ ││ │└─┴─┴───┘││
│ │└─┴─────────┘│
└───────────┴─────────────┘

┌───────────┬─────────────┐
│┌─┬─┬─────┐│┌─┬─────────┐│
││0│0│0 1 0│││1│┌─┬─┬───┐││
│└─┴─┴─────┘││ ││0│0│0 1│││
│ ││ │└─┴─┴───┘││
│ │└─┴─────────┘│
└───────────┴─────────────┘

## Match `≡`

Match is like equality but for non-scalar things. You can be forgiven to think that if you look for a particular vector in a vector-of-vectors, you should be able to use equality, but that doesn't work:

In [2]:
data←'aaa' 'bbb' 'ccc' 'ddd' 'ccc' 'aaa' 'ccc' 'bbb'
'ccc' = data  ⍝ LENGTH ERROR!

LENGTH ERROR
      'ccc'=data  ⍝ LENGTH ERROR!
           ∧


Instead we need either jot-match-each `∘≡¨`, or enclose-match-each:

In [6]:
⍸'ccc'∘≡¨data
⍸(⊂'ccc')≡¨data

3 5 7

3 5 7

## Index-of `A⍳B`

Another way we can find the index of things is via _index of_, dyadic `⍳`, which we have actually met before:

In [14]:
data⍳2

12

Dyadic iota finds the _first_ index - or 1 plus the last index if not found. This has the nice feature that we can feed it an array to the right:

In [18]:
data⍳2 3

12 21

Which, if we want to do it with equality requires a bit more dexterity -- but this approach would also spot multiples:

In [23]:
⍸∨⌿2 3 ∘.= data

12 21

What about if we crank the rank a bit?

In [26]:
⊢mat←4 4⍴2 15 16 14 11 9 12 10 13 1 7 5 4 8 6 3

2 15 16 14
11 9 12 10
13 1 7 5
 4 8 6 3

In [29]:
⊢mask←7=mat
⍸mask

0 0 0 0
0 0 0 0
0 0 1 0
0 0 0 0

┌───┐
│3 3│
└───┘

Gotta love APL. 

## Find `⍷`

There is of course also a glyph that's called _find_, `⍷`. Find locates the start-points of subsequences:

In [1]:
'ana' ⍷ 'banana'

done

Rebuilding user command cache... done

0 1 0 1 0 0

That combines nicely with `⍸` as a tacit atop to give a bit of an idiom to commit to memory:

In [2]:
substr ← ⍸⍷
'ana' substr 'banana'

2 4

We can use find to locate arrays-in-arrays of higher ranks, too:

In [7]:
⊢needle ← 2 2⍴0 1 1 0
⊢haystack ← 4 4⍴0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0

┌→──┐
↓0 1│
│1 0│
└~──┘

┌→──────┐
↓0 1 0 0│
│1 0 0 1│
│0 0 1 0│
│0 1 0 0│
└~──────┘

In [8]:
needle ⍷ haystack

┌→──────┐
↓1 0 0 0│
│0 0 1 0│
│0 1 0 0│
│0 0 0 0│
└~──────┘

The 1s represent the top-left corners of the 'needle' array in 'haystack':

![find1](./IMG/find1.png)

and we can pick out the actual coordinates using the same idiom we used for `substring` above:

In [10]:
needle (⍸⍷) haystack

┌→──────────────────┐
│ ┌→──┐ ┌→──┐ ┌→──┐ │
│ │0 0│ │1 2│ │2 1│ │
│ └~──┘ └~──┘ └~──┘ │
└∊──────────────────┘

## Regular expessions

Dyalog supports regular expressions through the most excellent [PCRE](http://help.dyalog.com/18.0/index.htm#Language/Appendices/PCRE%20Regular%20Expression%20Details.htm) engine. Dyalog's docs on the topic can be found [here](http://help.dyalog.com/18.0/index.htm#Language/System%20Functions/r.htm), and an APL Orchard cultivation was dedicated to the [topic](https://chat.stackexchange.com/rooms/52405/conversation/lesson-24-r-and-s), too. 

How regexes themselves work is beyond the scope of this book, but let's take a look at how they're integrated in Dyalog.

Two system operators, `⎕S` and `⎕R`, implement regex search and replace respectively. They're both dyadic operators, taking regular expression(s) to the left and transformation(s) to the right. The derived function can be applied to text data.

Here's a simple example. Using a transformation string of `&`, `⎕S` returns a vector of what was matched:

In [20]:
('ll.\sw'⎕S'&')'hello world well  worn'

┌→────────────────┐
│ ┌→────┐ ┌→────┐ │
│ │llo w│ │ll w│ │
│ └─────┘ └─────┘ │
└∊────────────────┘

The left operand can be a nested vector of regexes:

In [24]:
'he' 'wo' 'll'⎕S'&' ⊢ 'hello world well  worn'

┌→─────────────────────────┐
│ ┌→─┐ ┌→─┐ ┌→─┐ ┌→─┐ ┌→─┐ │
│ │he│ │ll│ │wo│ │ll│ │wo│ │
│ └──┘ └──┘ └──┘ └──┘ └──┘ │
└∊─────────────────────────┘

The right operand can be a function, too. This is where it gets a bit ~hairy~ flexible. We could have written the above as:

In [4]:
'he' 'wo' 'll'⎕S{⍵.Match}'hello world well  worn'

┌──┬──┬──┬──┬──┐
│he│ll│wo│ll│wo│
└──┴──┴──┴──┴──┘

In other words, the right operand function gets passed a _namespace_ representing the match at that point. In this case, the `⍵.Match` holds "what the current regex matched" - the same as the magic transformation short-hand `'&'`. We need to rely on a right operand function to make the most of regex capture groups. It's a bit of a shame that we don't have Perl's default capture group behaviour, or Python's `re.findall()`, but we can achieve the same thing, succumbing to an unusually verbose operand:

In [26]:
groups←{⍵.(1↓Lengths↑¨Offsets↓¨⊂Block)} ⍝ This is worth remembering!

In [27]:
'(a{2,}|b{2,})'⎕S groups 'aaabababbbbbaaaa'

┌→─────────────────────────────────┐
│ ┌→──────┐ ┌→────────┐ ┌→───────┐ │
│ │ ┌→──┐ │ │ ┌→────┐ │ │ ┌→───┐ │ │
│ │ │aaa│ │ │ │bbbbb│ │ │ │aaaa│ │ │
│ │ └───┘ │ │ └─────┘ │ │ └────┘ │ │
│ └∊──────┘ └∊────────┘ └∊───────┘ │
└∊─────────────────────────────────┘

When using a function operand, we have the opportunity to apply the full might of APL to the matches. In the example above we picked out stretches of 2 or longer of `a` or `b`. How long were they?

In [31]:
'(a{2,}|b{2,})'⎕S{≢⊃⍵.(1↓Lengths↑¨Offsets↓¨⊂Block)}'aaabababbbbbaaaa'

┌→────┐
│3 5 4│
└~────┘

The regex replacement operator, `⎕R`, operates much in the same way, but here the right operand represents a substitution into the right argument:

In [32]:
'a.a'⎕R'AXA'⊢'abababadabaaba'

┌→─────────────┐
│AXAbAXAdAXAAXA│
└──────────────┘

In the right operand we can refer to capture groups as backslash-X where X is a number from 1 to 9, following the Perl convention:

In [33]:
'a(.)a'⎕R'A\1A'⊢'abababadabaaba' ⍝ \1 is the first capture group

┌→─────────────┐
│AbAbAbAdAbAAbA│
└──────────────┘

A handy trick is to split strings based on a regular expression, removing the separators in the process. This is tricker than you might guess in Dyalog. Here's what [APL Cart](https://aplcart.info/?q=regex%20split#) suggests:

In [1]:
rsplit←{(⊢/¨r)↓¨⍵⊂⍨(⍳≢⍵)∊1+⊃¨r←(⍺,'|^')⎕S 0 1⊢⍵} ⍝ APL Cart

Rebuilding user command cache... done

Rebuilding user command cache... done

In [2]:
'\d+' rsplit 'aaaa6666bbb1cccc999eee87'

┌────┬───┬────┬───┬┐
│aaaa│bbb│cccc│eee││
└────┴───┴────┴───┴┘

Note the empty segment at the end.